## Final Notebook|| CASCADE CUP '22


### 1. Data Cleaning

In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

In [2]:
data_train=pd.read_csv("data/train.csv")
data_train=shuffle(data_train,random_state=0)

In [3]:
train_labels=data_train["cancelled"]
data_train=data_train.drop(['pickup_time','accept_time','allot_time','order_date','order_time','reassignment_reason',
                            'reassigned_order','rider_id','reassignment_method','cancelled_time','undelivered_orders',
                            'delivered_time','cancelled','order_id'],axis=1)
data_train.to_numpy()
data_train=np.nan_to_num(data_train)
train_labels.to_numpy()
train_examples=data_train

In [13]:
X = train_examples
y = train_labels

X_train, X_rem, y_train, y_rem = train_test_split(X,y, train_size=0.8)
X_valid, X_test, y_valid, y_test = train_test_split(X_rem,y_rem, test_size=0.5)
BATCH_SIZE = int((X_train.shape[0])/1000)

###  2. TWEAK THE MODEL

In [14]:
def get_basic_model():
  model = tf.keras.Sequential([
    tf.keras.layers.InputLayer(input_shape=(6,)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(60, activation='relu'),
    tf.keras.layers.Dense(1)
  ])
  #opt = tf.keras.optimizers.SGD(lr=0.0002)
  model.compile(optimizer='adam',
                loss='binary_crossentropy',
                metrics=['accuracy'])
  return model

In [15]:
model = get_basic_model()
model.fit(X_train,y_train, epochs=1, batch_size=BATCH_SIZE,validation_data=(X_valid,y_valid))

  20/1000 [..............................] - ETA: 5s - loss: 0.8129 - accuracy: 0.5458

2022-02-05 11:20:03.461716: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


1000/1000 [==============================] - ETA: 0s - loss: 0.1144 - accuracy: 0.9682

2022-02-05 11:20:07.974845: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


1000/1000 [==============================] - 5s 5ms/step - loss: 0.1144 - accuracy: 0.9682 - val_loss: 0.0691 - val_accuracy: 0.9878


In [16]:
model.evaluate(X_test,y_test)

  46/1407 [..............................] - ETA: 4s - loss: 0.0719 - accuracy: 0.9878

2022-02-05 11:20:09.871925: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


1407/1407 [==============================] - 5s 3ms/step - loss: 0.0670 - accuracy: 0.9882


[0.0669855996966362, 0.9882222414016724]

### 3. CSV SUBMISSION

In [18]:
test=pd.read_csv("data/test.csv")
test_id=test["order_id"]
test=test.drop(['accept_time','allot_time','order_date','order_time','reassignment_reason',
                            'reassigned_order','rider_id','reassignment_method','undelivered_orders',
                            'order_id'],axis=1)
test.to_numpy()
test=np.nan_to_num(test)

In [19]:
ans=model.predict(test)

2022-02-05 11:20:40.031373: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


In [20]:
sum(ans)/len(ans)

array([0.01261097])

In [21]:
data=pd.DataFrame()
ans=np.where(ans > 0.5, 1, 0)
data.insert(0,"order_id",test_id)
data.insert(1,"cancelled",ans)


In [22]:
data.head()

,order_id,cancelled
0,130231,0
1,130232,0
2,130233,0
3,130234,0
4,130235,0


In [23]:
data.to_csv("data/model1_22.csv",index=False)